![NASA logo](https://upload.wikimedia.org/wikipedia/commons/thumb/e/e5/NASA_logo.svg/110px-NASA_logo.svg.png) ![IBM Research logo](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSwHxsDwxcOHsQUD2pghQ32j90pzsZLcOujpGCyU1yE&s)

# Geospatial Foundation Model: Flood mapping fine-tuning

This is an example of how to fine-tune a model for flood mapping from HLS data using the IBM Geospatial Foundation models as a starting point.  

To run a fine-tuning experiment for flood mapping we will use the MMSegmentation library (https://github.ibm.com/GeoFM-Finetuning/mmsegmentation) to fine-tune a model starting from the geospatial foundation model trained on HLS data.

The following notebook assumes that you project files are placed in folder on the shared volume in the following folder structure:
```
configs                   - folder to place experiment configuration files
fine-tune-checkpoints     - folder where training outputs will be generated
GFM-Models                - folder containing the checkpoint files from the pre-trained GFM
inference                 - folder where we will carry out our inference tasks
training_data             - folder containing the training dataset (including labels and test/train splits etc)
```

You then create you configuration script, before submitting to the cluster to run.  The notebook will then guide you to: 
* monitor and visualise the training, 
* run the test tasks
* use the trained model for local inference.


In [ ]:
import json
import pandas as pd
import glob
import matplotlib.pyplot as plt
import os
import subprocess
from pprint import pprint
from dotenv import load_dotenv
import datetime
import string
import sys

sys.path.append('../')
import geoft

# Load environment variables
load_dotenv()

# Grab cluster details
login_url, namespace, path_to_shared_volume = geoft.get_cluster_details()

# Create S3 client (for pulling data and model weights)
aws_access_key_id = '19ef4682acca46c4a8c2405d34b4fc3f'
aws_access_key_secret = 'd57f50701a34d33fcfadd95411174fe0390f8234f83863a0'

s3 = geoft.create_s3_client(aws_access_key_id, aws_access_key_secret)

# S3 bucket where data and model weights reside
bucket_name = "nasa-gfm-summer-school"

In [ ]:
#------- Define the project name you wish to use

project_name = "flood"

## Connect to cluster for task submission

The first thing you need to do in order to submit a training job to the cluster is login to the cluster.  This will only need to be done once per 24 hours.

Run the cell below (`login_url`), and click on the generated url.

Authenticate, then copy and paste the `oc login` command into the cell below (with `%%sh` at the top) and this will log you in to the cluster and allow you submit and monitor jobs.

In [ ]:
login_url

In [ ]:
%%sh
# <Paste oc command here>

## Project setup

If we are starting a new fine-tuning project, we can create a new set of folders and download the training data+labels and the pre-train foundation model weights.  We create the folder structure described above, then pull the data and weights from an S3 bucket.

In [ ]:
#------- Create project folder structure 
geoft.create_project_folders(project_name)

In [ ]:
#------- Download the pre-trained model weights
model_name = 'epoch-916-loss-0.0779.pt' # best for flood mapping
s3.download_file(bucket_name, 'gfm-models/' + model_name, path_to_shared_volume + project_name + '/gfm-models/' + model_name)


In [ ]:
#------- Download the training data

dataset = 'sen1floods11'

training_data_path = path_to_shared_volume + project_name + '/training-data/'

# Download training data
subfolder = 'dataset/S2HandGeoDN/'
geoft.download_s3_dir(dataset + '/' + subfolder, training_data_path, bucket_name, client=s3)

# Download training data labels
subfolder = 'dataset/LabelHand/'
geoft.download_s3_dir(dataset + '/' + subfolder, training_data_path, bucket_name, client=s3)

# Download training data splits
subfolder = 'data_splits/'
geoft.download_s3_dir(dataset + '/' + subfolder, training_data_path, bucket_name, client=s3)

# Download data for batch inference
inference_data_path = path_to_shared_volume + project_name + '/inference/'
subfolder = 'inference/'
geoft.download_s3_dir(dataset + '/' + subfolder, inference_data_path, bucket_name, client=s3, key_dir_levels=0)


## Creating fine-tuning configuration

![Fine-tune architecture](../images/finetune_arch.png)

### Brief introduction to the hyperparameters we will adapt as part of this session
**Loss function**:
    Both tasks we will solve as part of this exercise are binary semantic segmentation task (e.g., pixelwise classification of flood vs. background). There will be two available loss functions for the task:
| Loss functions | Description | Code |
| -------------- | ----------- | ---- |
| CrossEntropyLoss | is sensitive to class imbalance but very general and a good choice for an initial training | `type='CrossEntropyLoss', use_sigmoid=False, loss_weight=1, class_weight=[0.15, 0.85, 0]` |
| DiceLoss | is invariant against class imbalance but tends to be more sensitive to other hyperparameters | `type='DiceLoss', use_sigmoid=False, loss_weight=1` |

**Weighting classes** in the loss function:
    As described above some loss functions (like CE loss) are sensitive to class imbalance. We can counter class imbalance by weighting the classes in the loss. For example, for flood mapping ~5-10% of the pixels represent parts of flood events while the rest is background. To meet the class imbalance, we can set the class weight of the flood class to, e.g., 90%, while the background class will be assigned a class weight of 10%. The data for flood mapping contains a third class for invalid pixels (including for example cloud coverage). The class weight for this class will be set to zero. For segmentation on burn scars (see other notebook), there are only two classes, so only two class weights need to be specified and the weight for the third class can be removed.
    
Example of Cross Entropy Loss options for flood mapping:
| Weight land class | Weight water class | Weight cloud class | Code | 
| ------------------ | ----------------- | ------------------ | ---- |
| 0.3 | 0.7 | 0.0 | `[0.7, 0.3, 0]` |
| 0.1 | 0.9 | 0.0 | `[0.9, 0.1, 0]` |

<!--     * cross entropy loss with weight water class = 0.7, weight land class = 0.3, weight cloud class = 0.0,
    * cross entropy loss with weight water class = 0.9, weight land class = 0.1, weight cloud class = 0.0
     -->
**Batch size**: Defines the number of training examples in a single forward/backward pass (i.e., how many images the model sees as part of one iteration). Batch sizes are typically powers of 2 to facilitate GPU computations. Options in this exercise are: `2`, `4`, `8`, `16`, `32`
<!--     * 2
    * 4
    * 8
    * 16
    * 32
 -->
**Learning rate**: Defines how much we want the model to change in response to the estimated error each time the model weights are updated. Options: `6e-4`, `6e-5`, `6e-6`

**Auxiliary head**: To stabilize the finetuning process, the model not only includes an encoder and a decode head for segmentation, but also an auxiliary head. This part of the architecture helps to make the model more robust during finetuning. You can add and remove the auxiliary head using the boolean option: `aux_head=True`, `aux_head=False`

**Depth of the decoder**: Generally, the decoder is quite light-weight compared to the GeoFM encoder. A default choice would be one or two layers of convolutions. Increasing this value will result in more parameters that the model can leverage to adapt to the downstream task -- at the cost of heavier computations (finetuning will take more time!). Options: `decode_head_conv = 1`, `decode_head_conv = 2`

**Number of epochs**: Deep neural networks are typically require a certain number of epochs to converge. For example, in our experiments, we observed that the finetuning for flood mapping achieves a desirable level of fitness after ~40-50 epochs. *Please do not extend the number of epochs to more than 50 epochs to have a managable time for computations. :-)*

### Setting up your experiment
Now to set up your experiment populate the dictionary below with the options you wish to chose (based on the description above and discussions).  Don't edit the `gfm_ckpt`, `num_epochs`, `number_training_files` or `project_name`.

You generate your config which places the options you have chosen into a configuration file, which you can then view using the next cell.


In [ ]:
conf = {'gfm_ckpt': 'epoch-916-loss-0.0779.pt',
        'loss_function': '''type='CrossEntropyLoss', use_sigmoid=False, loss_weight=1, class_weight=[0.15, 0.85, 0]''',
        'batch_size': '2',
        'learning_rate': '6e-5',
        'aux_head': 'True',
        'decode_head_conv': '1',
        'num_epochs': 15,
        'number_training_files': 252,
        'project_name': project_name}


In [ ]:
experiment_name, experiment_filepath = geoft.generate_config(project_name, conf, "flood_config.py.template")

In [ ]:
geoft.view_config(experiment_filepath)


## Submitting fine-tuning job to run
Now we have the configuration script ready, we can just send it to the cluster to run.  The next cell will submit the job to the cluster using TorchX.  This will spin up a now pod/container where the fine-tuning will run.


In [ ]:
mcad_id = geoft.submit_tune(project_name,
                namespace,
                experiment_name,
                image='quay.io/bedwards-ibm/mmsegmentation-geo:latest',
                num_gpus=1,
                memory_mb=24000)

print(mcad_id)

## Monitoring training job
Once you have submitted the job to the cluster, we can monitor it using the following commands.


In [ ]:
%%sh
torchx list -s kubernetes_mcad

In [ ]:
check_log_cmd = '''torchx log ''' + str(mcad_id) +  ''' | tail -n20'''
os.system(check_log_cmd)


## Viewing the training metrics

Now that we have run (or at least are running) the experiment, we can view the training metrics.  To do this we will load the log file and extract the metrics to a dataframe (`val_df`).

In [ ]:
train_df, val_df = geoft.load_tune_metrics(project_name, experiment_name)

In [ ]:
plt.figure().set_figwidth(15)
plt.subplot(1, 2, 1)
plt.plot(train_df.index, train_df.loss, '-r');
plt.ylabel('Training Loss');

plt.subplot(1, 2, 2)
plt.plot(train_df.index, train_df.loss_val, '-b');
plt.ylabel('Validation Loss');

## Test output model

In [ ]:
test_mcad_id = geoft.submit_test(project_name,
                        namespace,
                        experiment_name,
                        checkpoint='latest.pth',
                        num_gpus=1,
                        memory_mb=8000)

In [ ]:
check_log_cmd = '''torchx log ''' + str(test_mcad_id) +  ''' | tail -n20'''
os.system(check_log_cmd)

In [ ]:
test_metrics = geoft.get_test_metrics(project_name, experiment_name)

## Running inference using the trained model

Once we have a trained model, we can use it to run inference on other images.

In [ ]:
infer_mcad_id = geoft.submit_inference(project_name,
                namespace,
                experiment_name,
                checkpoint='latest.pth',
                image='quay.io/bedwards-ibm/mmsegmentation-geo:latest',
                num_gpus=1,
                memory_mb=8000)

## Visualizing the predicitons

In [ ]:
!pip install rasterio folium

In [ ]:
import folium
import folium.plugins as plugins
import numpy as np
import rasterio
from rasterio.plot import show

def colorize(array, cmax, cmin=0, cmap="rainbow"):
    """Converts a 2D numpy array of values into an RGBA array given a colour map and range.
    Args:
        array (ndarray):
        cmax (float): Max value for colour range
        cmin (float): Min value for colour range
        cmap (string): Colour map to use (from matplotlib colourmaps)
    Returns:
            rgba_array (ndarray): 3D RGBA array which can be plotted.
    """
    normed_data = (array - cmin) / (array.max() - cmin)
    cm = plt.cm.get_cmap(cmap)
    return cm(normed_data)

def load_s2_data(data1, index, quantmin=0.1, quantmax=0.9, target = {'Red':0.665, 'Green': 0.560, 'Blue': 0.460}):
    
    data1 = np.where((data1 ==  -9999.0) |(data1 ==  100) , np.nan, data1)
    mean = np.nanmean(data1)
    data1 = target[index]*data1/mean
    data1 = np.where((data1 <  0) , 0, data1)
    data1 = np.where((data1 >  1) , 1, data1)
        
    return data1


In [ ]:
inference_files = sorted(glob.glob('/opt/app-root/src/data/' + project_name + '/inference/*.tif'))
inference_files

In [ ]:
filenum=1
original_file = inference_files[filenum]
predict_file = inference_files[filenum].replace('/inference','/inference/pred/' + experiment_name).replace('.tif','_pred.tif')

# Load the original image layer
with rasterio.open(original_file) as src:
    redArray = src.read(1)
    redArray = load_s2_data(redArray/10000, 'Red', quantmin=0.01, quantmax=0.99)*0.5
    greenArray = src.read(2)
    greenArray = load_s2_data(greenArray/10000, 'Green', quantmin=0.01, quantmax=0.99)*0.5
    blueArray = src.read(3)
    blueArray = load_s2_data(blueArray/10000, 'Blue', quantmin=0.01, quantmax=0.99)*0.5
    bounds = src.bounds
    nd = src.nodata
    midLat = (bounds[3] + bounds[1]) / 2
    midLon = (bounds[2] + bounds[0]) / 2
    im_rgb = np.moveaxis(np.array([redArray,greenArray,blueArray]), 0, -1)/2048


# Create the map
m = folium.Map(location=[midLat, midLon], tiles='openstreetmap', max_zoom=22)

# Add the prediciton layer to the map
with rasterio.open(predict_file) as src:
    dataArray = src.read(1)
    bounds = src.bounds
    nd = src.nodata

# cmax = np.max(dataArray)
cmax = 1000
dataArrayMasked = np.ma.masked_where(dataArray == nd, dataArray)
dataArrayMasked = np.ma.masked_where(dataArray == 0, dataArrayMasked)
imc = colorize(dataArrayMasked, cmax, cmin=0, cmap="viridis")

# Add the layers to the map
pred = folium.raster_layers.ImageOverlay(imc, [[bounds[1], bounds[0]], [bounds[3], bounds[2]]], name="Prediction", opacity=0.8)
orig = folium.raster_layers.ImageOverlay(im_rgb, [[bounds[1], bounds[0]], [bounds[3], bounds[2]]], name="Original image", opacity=1.0)

orig.add_to(m)
pred.add_to(m)

folium.LayerControl().add_to(m)
m.fit_bounds(bounds)

m  